In [2]:
import tensorflow as tf
import pickle
import numpy as np
import os

CIFAR_DIR = './data/cifar-10-batches-py/'
print(os.listdir(CIFAR_DIR))

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [3]:
def load_data(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='latin1')
        return data['data'], data['labels']
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()

    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception('have no more examples')
        if end_indicator > self._num_examples:
            raise Exception('batch size is larger than all data')
        batch_data = self._data[self._indicator : end_indicator]
        batch_labels = self._labels[self._indicator : end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]
train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)
batch_data, batch_labels = train_data.next_batch(10)

In [5]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

# 卷积神经网络的输入是image
# 先将一维的xreshape为多通道image
x_image = tf.reshape(x, [-1, 3, 32, 32])
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# 32 * 32
# 通过一个卷积层的输出可以称为feature map,神经元图，输出图像
conv1 = tf.layers.conv2d(x_image,
                        32,# output channels num,
                        (3, 3), # kernel size
                        padding='same',
                        activation=tf.nn.relu,
                        name='conv1')
# padding规则为same，所以通过卷积层后的图像size没有改变
# 而且output channels num是32，每个通道上引入了32个卷积核，相当于提取了32个特征
# 这里使用max pooling，输出图像size会减半
# pooling后的size是n / kernel size，而且stride = kernel size
# 16 * 16
pooling1 = tf.layers.max_pooling2d(conv1,
                                  (2, 2),# kernel size
                                  (2, 2),# stride 
                                  name='pooling1')


conv2 = tf.layers.conv2d(pooling1,
                        32,
                        (3, 3),
                        padding='same',
                        activation=tf.nn.relu,
                        name='conv2')

# 8 * 8
pooling2 = tf.layers.max_pooling2d(conv2,
                                  (2, 2),
                                  (2, 2),
                                  name='pooling2')

conv3 = tf.layers.conv2d(pooling2,
                        32,
                        (3, 3),
                        padding='same',
                        activation=tf.nn.relu,
                        name='conv3')

# 4 * 4 * 32
pooling3 = tf.layers.max_pooling2d(conv3,
                                  (2, 2),
                                  (2, 2),
                                  name='pooling3')

# 最后一层是全连接层
flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

predict = tf.argmax(y_, 1)
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [6]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss, accuracy, train_op],
                                       feed_dict={x:batch_data, y:batch_labels})
        if (i + 1) % 500 == 0:
            print('[Train] step: %d, loss: %4.5f, acc: %4.5f' % (i + 1, loss_val, acc_val))
        if (i + 1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_data_batch, test_data_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict={x:test_data_batch, y:test_data_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] step: %d, acc: %4.5f' % (i + 1, test_acc))

[Train] step: 500, loss: 1.23910, acc: 0.65000
[Train] step: 1000, loss: 1.19864, acc: 0.50000
[Train] step: 1500, loss: 1.20486, acc: 0.60000
[Train] step: 2000, loss: 1.28998, acc: 0.50000
[Train] step: 2500, loss: 1.37554, acc: 0.40000
[Train] step: 3000, loss: 1.07068, acc: 0.65000
[Train] step: 3500, loss: 0.86282, acc: 0.65000
[Train] step: 4000, loss: 1.04822, acc: 0.60000
[Train] step: 4500, loss: 0.90431, acc: 0.65000
[Train] step: 5000, loss: 0.92070, acc: 0.55000
[Test] step: 5000, acc: 0.67650
[Train] step: 5500, loss: 0.95775, acc: 0.65000
[Train] step: 6000, loss: 0.70703, acc: 0.75000
[Train] step: 6500, loss: 0.92188, acc: 0.65000
[Train] step: 7000, loss: 0.65955, acc: 0.70000
[Train] step: 7500, loss: 1.01059, acc: 0.70000
[Train] step: 8000, loss: 0.89973, acc: 0.65000
[Train] step: 8500, loss: 1.29219, acc: 0.60000
[Train] step: 9000, loss: 0.79009, acc: 0.80000
[Train] step: 9500, loss: 1.03958, acc: 0.60000
[Train] step: 10000, loss: 0.62582, acc: 0.75000
[Test] s